In [1]:
!ls

Parser.ipynb  README.md  WNdb-3.0.tar.gz  dict	parsers


In [60]:
def createIndexDF(filePath):
    with open(filePath) as f:
        Indexdump = f.readlines()
    indexLines = []
    for i in Indexdump:
        if i[0] == " " or i[0] == ".":
            continue
        else:
            indexLines.append(i.strip())
    for i in range(len(indexLines)):
        splitLine = indexLines[i].split()
        if (len(splitLine) == 8):
            indexLines[i] = splitLine
            continue
        ls = []
        ls.extend(splitLine[0:4])
        p_cnt = int(ls[-1])
        ls.append(splitLine[4:4 + p_cnt])
        sense_cntIndex = 4 + p_cnt
        ls.append(splitLine[sense_cntIndex])
        tagsense_cntIndex = sense_cntIndex + 1
        ls.append(splitLine[tagsense_cntIndex])
        remainingStartIndex = tagsense_cntIndex + 1
        ls.append(splitLine[remainingStartIndex:])
        indexLines[i] = ls
    df = pd.DataFrame(indexLines, columns = ['lemma', 'pos', 'synset_cnt', 'p_cnt', 'ptr_symbol', 'sense_cnt', 'tagsense_cnt', 'synset_offset'])
    return df

In [54]:
import pandas as pd

In [16]:
with open('dict/index.adj') as f:
    Indexdump = f.readlines()

In [17]:
Indexdump

['  1 This software and database is being provided to you, the LICENSEE, by  \n',
 '  2 Princeton University under the following license.  By obtaining, using  \n',
 '  3 and/or copying this software and database, you agree that you have  \n',
 '  4 read, understood, and will comply with these terms and conditions.:  \n',
 '  5   \n',
 '  6 Permission to use, copy, modify and distribute this software and  \n',
 '  7 database and its documentation for any purpose and without fee or  \n',
 '  8 royalty is hereby granted, provided that you agree to comply with  \n',
 '  9 the following copyright notice and statements, including the disclaimer,  \n',
 '  10 and that the same appear on ALL copies of the software, database and  \n',
 '  11 documentation, including modifications that you make for internal  \n',
 '  12 use or for distribution.  \n',
 '  13   \n',
 '  14 WordNet 3.0 Copyright 2006 by Princeton University.  All rights reserved.  \n',
 '  15   \n',
 '  16 THIS SOFTWARE AND DATABA

In [50]:
indexLines = []
for i in Indexdump:
    if i[0] == " " or i[0] == ".":
        continue
    else:
        indexLines.append(i.strip())

In [51]:
indexLines

['0 a 1 1 & 1 1 02186132',
 '1 a 1 1 & 1 1 02186338',
 '10 a 1 1 & 1 1 02187296',
 '10-membered a 1 1 & 1 0 01503760',
 '100 a 1 1 & 1 1 02196107',
 '1000 a 1 1 & 1 1 02198752',
 '1000th a 1 1 & 1 0 02212473',
 '100th a 1 1 & 1 0 02209423',
 '101 a 1 1 & 1 0 02196211',
 '101st a 1 1 & 1 0 02209551',
 '105 a 1 1 & 1 0 02196333',
 '105th a 1 1 & 1 0 02209678',
 '10th a 1 1 & 1 0 02203373',
 '11 a 1 1 & 1 1 02187379',
 '110 a 1 1 & 1 0 02196439',
 '110th a 1 1 & 1 0 02209806',
 '115 a 1 1 & 1 0 02196543',
 '115th a 1 1 & 1 0 02209933',
 '11th a 1 1 & 1 1 02203500',
 '12 a 1 1 & 1 1 02187465',
 '120 a 1 1 & 1 0 02196657',
 '120th a 1 1 & 1 0 02210068',
 '125 a 1 1 & 1 0 02196769',
 '125th a 1 1 & 1 0 02210202',
 '12th a 1 1 & 1 0 02203629',
 '13 a 1 1 & 1 1 02187606',
 '130 a 1 1 & 1 0 02196891',
 '130th a 1 1 & 1 0 02210344',
 '135 a 1 1 & 1 0 02197007',
 '135th a 1 1 & 1 0 02210478',
 '13th a 1 1 & 1 1 02203763',
 '14 a 1 1 & 1 1 02187699',
 '140 a 1 1 & 1 0 02197130',
 '140th a 1 1 & 1 

In [52]:
for i in range(len(indexLines)):
    splitLine = indexLines[i].split()
    if (len(splitLine) == 8):
        indexLines[i] = splitLine
        continue
    ls = []
    ls.extend(splitLine[0:4])
    p_cnt = int(ls[-1])
    ls.append(splitLine[4:4 + p_cnt])
    sense_cntIndex = 4 + p_cnt
    ls.append(splitLine[sense_cntIndex])
    tagsense_cntIndex = sense_cntIndex + 1
    ls.append(splitLine[tagsense_cntIndex])
    remainingStartIndex = tagsense_cntIndex + 1
    ls.append(splitLine[remainingStartIndex:])
    indexLines[i] = ls

In [53]:
indexLines

[['0', 'a', '1', '1', '&', '1', '1', '02186132'],
 ['1', 'a', '1', '1', '&', '1', '1', '02186338'],
 ['10', 'a', '1', '1', '&', '1', '1', '02187296'],
 ['10-membered', 'a', '1', '1', '&', '1', '0', '01503760'],
 ['100', 'a', '1', '1', '&', '1', '1', '02196107'],
 ['1000', 'a', '1', '1', '&', '1', '1', '02198752'],
 ['1000th', 'a', '1', '1', '&', '1', '0', '02212473'],
 ['100th', 'a', '1', '1', '&', '1', '0', '02209423'],
 ['101', 'a', '1', '1', '&', '1', '0', '02196211'],
 ['101st', 'a', '1', '1', '&', '1', '0', '02209551'],
 ['105', 'a', '1', '1', '&', '1', '0', '02196333'],
 ['105th', 'a', '1', '1', '&', '1', '0', '02209678'],
 ['10th', 'a', '1', '1', '&', '1', '0', '02203373'],
 ['11', 'a', '1', '1', '&', '1', '1', '02187379'],
 ['110', 'a', '1', '1', '&', '1', '0', '02196439'],
 ['110th', 'a', '1', '1', '&', '1', '0', '02209806'],
 ['115', 'a', '1', '1', '&', '1', '0', '02196543'],
 ['115th', 'a', '1', '1', '&', '1', '0', '02209933'],
 ['11th', 'a', '1', '1', '&', '1', '1', '022035

In [57]:
df = pd.DataFrame(indexLines, columns = ['lemma', 'pos', 'synset_cnt', 'p_cnt', 'ptr_symbol', 'sense_cnt', 'tagsense_cnt', 'synset_offset'])

In [58]:
df

,lemma,pos,synset_cnt,p_cnt,ptr_symbol,sense_cnt,tagsense_cnt,synset_offset
0,0,a,1,1,&,1,1,02186132
1,1,a,1,1,&,1,1,02186338
2,10,a,1,1,&,1,1,02187296
3,10-membered,a,1,1,&,1,0,01503760
4,100,a,1,1,&,1,1,02196107
...,...,...,...,...,...,...,...,...
21462,zygomorphous,a,1,1,;,1,0,02374697
21463,zygotic,a,1,2,"[\, +]",1,0,[02882275]
21464,zymoid,a,1,1,\,1,0,03146237
21465,zymolytic,a,1,2,"[\, +]",1,0,[03000447]
